In [170]:
import pandas as pd
import re
from mlx_llm.model import create_model
from transformers import BertTokenizer
import mlx.core as mx
from mlx.nn import losses as loss
import jax.numpy as jnp
from mlx_embedding_models.embedding import EmbeddingModel
import chromadb

In [171]:
df_products = pd.read_csv('./Data/products.csv')

In [172]:
chroma_client = chromadb.PersistentClient('./vectordb')

In [173]:
collection = chroma_client.get_or_create_collection("telemerecsys")

In [175]:
# clean the data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\s+',' ',text)
    return text

df_products['cleaned_name'] = df_products['name'].apply(lambda x: clean_text(x))

# create the embedding model
model = EmbeddingModel.from_registry("bge-small")

# define the get embeddings function
def get_embeddings(text):
    return model.encode([text])[0]

# get the embeddings for each product
df_products['embeddings'] = df_products['cleaned_name'].apply(get_embeddings)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 73.28it/s]


In [180]:
type(df_products['embeddings'].to_list()[0])

numpy.ndarray

In [182]:
collection.add(
    embeddings=[emb.tolist() for emb in df_products['embeddings'].to_list()],
    metadatas=[{"price": price} for price in df_products['price']],
    documents=[doc for doc in df_products['cleaned_name']],
    ids=[str(i) for i in range(len(df_products))]
)

In [163]:
df_products.to_csv("cleaned_products_data.csv", index=False)